In [1]:
from utils.data_preprocess import *
import pandas as pd
import numpy as np
import tensorflow as tf

In [116]:
df = pd.read_csv('ETHBTC-5m-data.csv')

In [117]:
data = df.set_index('Timestamp').iloc[:, :5]

In [120]:
data['Log_return'] = np.log(data.Close).diff()
data['Log_return+1'] = data.Log_return.shift(-24)
data['STD'] = data.Close.rolling(10).std()

In [114]:
data = data.dropna()

In [121]:
data.head()

,Open,High,Low,Close,Volume,Log_return,Log_return+1,STD
Timestamp,,,,,,,,
2017-07-14 04:00:00,0.080000,0.080000,0.080000,0.080000,0.726,NaN,0.000714,NaN
2017-07-14 04:05:00,0.080000,0.080001,0.080000,0.080001,3.347,0.000012,0.008459,NaN
2017-07-14 04:10:00,0.080001,0.086400,0.080001,0.086400,4.679,0.076949,0.009090,NaN
2017-07-14 04:15:00,0.085289,0.085620,0.085128,0.085128,53.431,-0.014832,-0.000851,NaN
2017-07-14 04:20:00,0.085274,0.086000,0.085274,0.086000,5.576,0.010191,0.002276,NaN


In [109]:
# data['Trend'] = np.where(data['Log_return+1'] >= data['Log_return']*(1 + 0.55*data['STD']), 1,
#                          (np.where(data['Log_return+1'] <= data['Log_return']*(1 - 0.55*data['STD']), -1, 0)))